In [14]:
import os

from dotenv import load_dotenv

print("Generating data")

Generating data


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os
import pandas as pd

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [6]:
df = pd.read_csv('public/economy_delivery_data/real_data.csv')
df.head()

,Order ID,Customer ID,Platform,Order Date & Time,Delivery Time (Minutes),Product Category,Order Value (INR),Customer Feedback,Service Rating,Delivery Delay,Refund Requested
0,ORD000001,CUST2824,JioMart,19:29.5,30,Fruits & Vegetables,382,"Fast delivery, great service!",5,No,No
1,ORD000002,CUST1409,Blinkit,54:29.5,16,Dairy,279,Quick and reliable!,5,No,No
2,ORD000003,CUST5506,JioMart,21:29.5,25,Beverages,599,Items missing from order.,2,No,Yes
3,ORD000004,CUST5012,JioMart,19:29.5,42,Beverages,946,Items missing from order.,2,Yes,Yes
4,ORD000005,CUST4657,Blinkit,49:29.5,30,Beverages,334,"Fast delivery, great service!",5,No,No


In [4]:
import random
from faker import Faker

# Define the possible customer types
customer_types = ["Vip", "Bronze", "Sliver", "Gold", "Diamond"]

# Initialize Faker
fake = Faker('vi_VN')

# Create a dictionary to hold the fake data for each unique CustomerID
customer_info = {}

for cid in df['Customer ID'].unique():
    Faker.seed(random.randint(0, 100000))
    # Generate fake data
    country = fake.country()
    country_code = fake.country_code()
    # Note: Faker does not tie city to a specific country, so we simply generate a random city.
    city = fake.city()
    address = fake.address().replace("\n", ", ")  # Remove newlines for a single-line address.
    # Generate a fake name; if you want the name to reflect a specific locale based on the country,
    # you could create a new Faker instance with a locale (if available) for that country.
    name = fake.name()
    ctype = random.choice(customer_types)
    post_code = fake.postcode()

    # Save the fake data in the dictionary
    customer_info[cid] = {
        "CustomerCountry": country,
        "CustomerAddress": address,
        "CustomerCity": city,
        "CustomerName": name,
        "CustomerType": ctype,
        "CustomerPostCode": post_code,
        "CustomerCountryCode": country_code,
        "Currency": fake.currency_code(),
        "JobCustomer": fake.job(),
        "Phone": fake.phone_number(),
    }


In [6]:
order_info = {}

for oid in df['Order ID'].unique():
    Faker.seed(random.randint(0, 100000))

    order_price = fake.pricetag()
    order_payment_methods = random.choice(['Visa', 'MasterCard', 'Exchange', 'Money'])
    order_delivery_methods = random.choice(['Fast', 'Standard', 'VIP'])
    order_total_items = random.randint(1, 10)

    order_info[oid] = {
        'OrderValue': order_price,
        'OrderPaymentMethod': order_payment_methods,
        'OrderDeliveryMethod': order_delivery_methods,
        'OrderTotalItems': order_total_items,
    }


In [7]:
customer_df = pd.DataFrame.from_dict(customer_info, orient='index').reset_index().rename(columns={'index': 'Customer ID'})

In [8]:
order_df = pd.DataFrame.from_dict(order_info, orient='index').reset_index().rename(columns={'index': 'Order ID'})

In [9]:
df = df.merge(customer_df, on='Customer ID', how='left')
df = df.merge(order_df, on='Order ID', how='left')

In [10]:
df.head()

,Order ID,Customer ID,Platform,Order Date & Time,Delivery Time (Minutes),Product Category,Order Value (INR),Customer Feedback,Service Rating,Delivery Delay,...,CustomerType,CustomerPostCode,CustomerCountryCode,Currency,JobCustomer,Phone,OrderValue,OrderPaymentMethod,OrderDeliveryMethod,OrderTotalItems
0,ORD000001,CUST2824,JioMart,19:29.5,30,Fruits & Vegetables,382,"Fast delivery, great service!",5,No,...,Diamond,301008,DZ,PGK,Quản lý sản xuất,04 8900389,"₫3,417.43",Exchange,VIP,10
1,ORD000002,CUST1409,Blinkit,54:29.5,16,Dairy,279,Quick and reliable!,5,No,...,Sliver,946454,TL,ZWD,Chuyên viên tín dụng,+84 66 7210134,₫43.77,MasterCard,Standard,5
2,ORD000003,CUST5506,JioMart,21:29.5,25,Beverages,599,Items missing from order.,2,No,...,Diamond,339969,MZ,CZK,Biên dịch viên,(08) 6196 7544,"₫7,953.23",Visa,VIP,7
3,ORD000004,CUST5012,JioMart,19:29.5,42,Beverages,946,Items missing from order.,2,Yes,...,Diamond,489220,IN,NOK,Nhân viên ngân hàng,07 8370 8078,"₫44,822.99",Money,Fast,1
4,ORD000005,CUST4657,Blinkit,49:29.5,30,Beverages,334,"Fast delivery, great service!",5,No,...,Vip,837755,UZ,MWK,Nhà phân tích dữ liệu,+84-96-289489,₫4.44,Visa,Fast,2


In [12]:
df.to_csv('public/economy_delivery_data/real_data_generate.csv', index=False)

In [8]:
# cut random 200 rows to become reference data
df1 = pd.read_csv('public/economy_delivery_data/real_data_generate.csv')
df1 = df1.sample(n=60, random_state=42)
df1.to_csv('public/economy_delivery_data/reference_data.csv', index=False)